In [3]:
import pandas as pd

df = pd.read_csv('Archive/summarized_results.csv')

# Sort the dataframe

df_sorted = df.drop(columns=['nn_model', 'number_of_clients'])
# Sort the dataframe by skew, dataset, and heterogeneity_class

# Define the order and bold the 'cornflqs' exp_type
exp_type_order = ['fedavg', 'fedprox', 'cfl', 'hcfl', 'fedgroup', 'ifca', 'srfca', 'cornflqs', 'oracle-centralized']
df_sorted['exp_type'] = pd.Categorical(df_sorted['exp_type'], categories=exp_type_order, ordered=True)
# Add a newline before each occurrence of 'fedavg' in the 'exp_type' column
df_sorted['exp_type'] = df_sorted['exp_type'].apply(lambda x: '\n' + x if x == 'fedavg' else x)
# Sort the dataframe by the defined order
df_sorted = df_sorted.sort_values(['skew', 'dataset', 'heterogeneity_class','exp_type'])
df_sorted.to_excel('results.xlsx', index=False)

In [4]:
df=pd.read_csv('/home/mbenali/Documents/Project/CFL-Evaluation-of-labels-and-quantity-skew/granular_results/noqs.csv')
# Define the order and bold the 'cornflqs' exp_type
df_sorted = df
exp_type_order = ['fedavg', 'fedprox', 'cfl', 'hcfl', 'fedgroup', 'ifca', 'srfca', 'cornflqs', 'oracle-centralized']
df_sorted['exp_type'] = pd.Categorical(df_sorted['exp_type'], categories=exp_type_order, ordered=True)
df_sorted['exp_type'] = df_sorted['exp_type'].apply(lambda x: '\n' + x if x == 'fedavg' else x)
df_sorted = df_sorted.sort_values(['skew', 'dataset', 'heterogeneity_class','exp_type'])
df_sorted.to_excel('noqs.xlsx', index=False)

In [5]:
df=pd.read_csv('/home/mbenali/Documents/Project/CFL-Evaluation-of-labels-and-quantity-skew/granular_results/qs1.csv')
df_sorted = df
exp_type_order = ['fedavg', 'fedprox', 'cfl', 'hcfl', 'fedgroup', 'ifca', 'srfca', 'cornflqs', 'oracle-centralized']
df_sorted['exp_type'] = pd.Categorical(df_sorted['exp_type'], categories=exp_type_order, ordered=True)
df_sorted['exp_type'] = df_sorted['exp_type'].apply(lambda x: '\n' + x if x == 'fedavg' else x)
df_sorted = df_sorted.sort_values(['skew', 'dataset', 'heterogeneity_class','exp_type'])
df_sorted.to_excel('qs1.xlsx', index=False)


In [6]:
df=pd.read_csv('/home/mbenali/Documents/Project/CFL-Evaluation-of-labels-and-quantity-skew/granular_results/qs2.csv')
# Define the order and bold the 'cornflqs' exp_type
exp_type_order = ['fedavg', 'fedprox', 'cfl', 'hcfl', 'fedgroup', 'ifca', 'srfca', 'cornflqs', 'oracle-centralized']
df_sorted = df
df_sorted['exp_type'] = pd.Categorical(df_sorted['exp_type'], categories=exp_type_order, ordered=True)
df_sorted['exp_type'] = df_sorted['exp_type'].apply(lambda x: '\n' + x if x == 'fedavg' else x)
df_sorted = df_sorted.sort_values(['skew', 'dataset', 'heterogeneity_class','exp_type'])
df_sorted.to_excel('qs2.xlsx', index=False)

In [185]:
def algo_ranking(dataset=None, skew=None, heterogeneity_class=None):
    # Load the CSV data into a pandas DataFrame
    df = pd.read_csv('Archive/summarized_results.csv')
    df.fillna('None', inplace=True)

    # Filter the dataframe based on the input parameters
    if dataset:
        df = df[df['dataset'] == dataset]
    if skew:
        if skew == 'None':
            df = df[df['skew'] == skew]
        elif skew == 'quantity-skew':
            df = df[df['skew'] != 'None']
        else:
            df = df[df['skew'] == skew]
    if heterogeneity_class:
        df = df[df['heterogeneity_class'] == heterogeneity_class]

    # Step 2: Clean the data
    # Remove the '\pm' and uncertainty values from the 'accuracy' column
    df['accuracy'] = df['accuracy'].str.split(' ').str[0].astype(float)
    # Replace 'n/a' with NaN for missing values

    # Convert metric columns to numeric
    metrics = ['accuracy', 'ARI', 'AMI', 'hom', 'cmplt', 'vm']
    df[metrics] = df[metrics].apply(pd.to_numeric, errors='coerce')

    # Concatenate 'dataset', 'heterogeneity_class', and 'skew' into one column named 'group'
    df['group'] = df['dataset'] + '_' + df['heterogeneity_class'] + '_' + df['skew']
    # Drop the original 'dataset', 'heterogeneity_class', and 'skew' columns
    df = df.drop(columns=['dataset', 'heterogeneity_class', 'skew'])

    # Step 3: Group the data by the new 'group' column
    grouped = df.groupby('group')

    # Step 4: Rank algorithms within each group for each metric
    def rank_algorithms(group):
        for metric in metrics:
            group[f'{metric}_rank'] = group[metric].rank(ascending=False, method='min')
        return group

    ranked_groups = grouped.apply(rank_algorithms)
    best = ranked_groups.groupby('exp_type')[[f'{metric}_rank' for metric in metrics]].median()
    best = best.sort_values(by='accuracy_rank')

    return best
# Example usage:


In [186]:
algo_ranking(skew='None')

/tmp/ipykernel_20558/2935412668.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'None' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna('None', inplace=True)
/tmp/ipykernel_20558/2935412668.py:42: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ranked_groups = grouped.apply(rank_algorithms)


,accuracy_rank,ARI_rank,AMI_rank,hom_rank,cmplt_rank,vm_rank
exp_type,,,,,,
hcfl,2.0,1.0,1.0,1.0,1.0,1.0
cfl,3.0,1.0,1.0,1.0,1.0,1.0
cornflqs,3.0,1.0,1.0,1.0,1.0,1.0
ifca,5.0,4.0,4.0,5.0,1.0,4.0
oracle-centralized,5.0,NaN,NaN,NaN,NaN,NaN
fedgroup,6.0,1.0,1.0,1.0,1.0,1.0
srfca,8.0,1.0,1.0,1.0,1.0,1.0
fedprox,9.0,NaN,NaN,NaN,NaN,NaN
fedavg,10.0,NaN,NaN,NaN,NaN,NaN


In [187]:
algo_ranking(skew='quantity-skew-type-1')

/tmp/ipykernel_20558/2935412668.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'None' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna('None', inplace=True)
/tmp/ipykernel_20558/2935412668.py:42: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ranked_groups = grouped.apply(rank_algorithms)


,accuracy_rank,ARI_rank,AMI_rank,hom_rank,cmplt_rank,vm_rank
exp_type,,,,,,
oracle-centralized,2.0,NaN,NaN,NaN,NaN,NaN
cornflqs,3.0,1.0,1.0,1.0,1.0,1.0
ifca,4.0,1.0,1.0,1.0,1.0,1.0
cfl,5.0,4.0,4.0,4.0,3.0,4.0
hcfl,5.0,5.0,5.0,4.0,6.0,4.0
fedgroup,7.0,6.0,7.0,6.0,7.0,7.0
srfca,8.5,3.0,3.0,3.5,1.0,3.0
fedprox,9.0,NaN,NaN,NaN,NaN,NaN
fedavg,10.0,NaN,NaN,NaN,NaN,NaN


In [188]:
algo_ranking(skew='quantity-skew-type-2')

/tmp/ipykernel_20558/2935412668.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'None' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna('None', inplace=True)
/tmp/ipykernel_20558/2935412668.py:42: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ranked_groups = grouped.apply(rank_algorithms)


,accuracy_rank,ARI_rank,AMI_rank,hom_rank,cmplt_rank,vm_rank
exp_type,,,,,,
hcfl,1.5,1.0,1.0,1.0,1.0,1.0
cornflqs,3.0,2.0,2.0,2.0,4.0,2.0
oracle-centralized,3.0,NaN,NaN,NaN,NaN,NaN
ifca,4.0,3.0,3.0,3.0,1.0,3.0
fedgroup,5.5,3.5,3.5,3.5,6.0,4.0
cfl,7.0,7.0,7.0,7.0,5.0,7.0
fedprox,8.0,NaN,NaN,NaN,NaN,NaN
fedavg,9.0,NaN,NaN,NaN,NaN,NaN
srfca,10.0,1.0,1.0,1.0,1.0,1.0


In [189]:
algo_ranking(skew='quantity-skew')

/tmp/ipykernel_20558/2935412668.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'None' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna('None', inplace=True)
/tmp/ipykernel_20558/2935412668.py:42: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ranked_groups = grouped.apply(rank_algorithms)


,accuracy_rank,ARI_rank,AMI_rank,hom_rank,cmplt_rank,vm_rank
exp_type,,,,,,
cornflqs,3.0,1.0,1.0,1.0,1.0,1.0
hcfl,3.0,1.0,1.0,1.0,1.0,1.0
oracle-centralized,3.0,NaN,NaN,NaN,NaN,NaN
ifca,4.0,1.5,1.5,1.5,1.0,1.5
cfl,6.0,5.0,5.5,5.0,4.5,5.5
fedgroup,6.0,5.0,5.0,5.0,6.0,5.0
fedprox,8.0,NaN,NaN,NaN,NaN,NaN
fedavg,9.0,NaN,NaN,NaN,NaN,NaN
srfca,9.0,1.0,1.0,1.0,1.0,1.0


In [192]:
algo_ranking()

/tmp/ipykernel_20558/2935412668.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'None' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna('None', inplace=True)
/tmp/ipykernel_20558/2935412668.py:42: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ranked_groups = grouped.apply(rank_algorithms)


,accuracy_rank,ARI_rank,AMI_rank,hom_rank,cmplt_rank,vm_rank
exp_type,,,,,,
hcfl,2.5,1.0,1.0,1.0,1.0,1.0
cornflqs,3.0,1.0,1.0,1.0,1.0,1.0
oracle-centralized,3.0,NaN,NaN,NaN,NaN,NaN
ifca,4.0,2.0,2.0,2.0,1.0,2.0
cfl,5.0,5.0,5.0,4.0,4.0,5.0
fedgroup,6.0,4.5,4.5,4.5,6.0,4.5
srfca,8.0,1.0,1.0,1.0,1.0,1.0
fedavg,9.0,NaN,NaN,NaN,NaN,NaN
fedprox,9.0,NaN,NaN,NaN,NaN,NaN
